In [ ]:
#o this just a basic part of using a indexing,retrievel and generation of an RAG 

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = <your-api-key>

In [ ]:
#the first part is indexing (representation of strings into embeddings for effective an easy search)
# this is just an exmaple for one query and one document but this is for study mainly we follow mant processes for embedding 
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)

#main chain of processes
#1)
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(#this bs4 is a lib which is used for scraping only the necessary content from the web to feed it to the llm now ffrom the blog title we onlt get the htl divisons mentioned wrt to the class here and we can also iterate /parse through the html loaded using beautifulSoup
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()#this loads the html docs from the link then convert it into docs format

#2)Spliiting into chunks while ensuring semantics
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

#3)Storing
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())
#retriever = vectorstore.as_retriever()
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})#so this is like  k nearest neighbours where you collect th k most semantically nearest vector chunk fromthe Chroma vectorbase
#docs = retriever.get_relevant_documents("What is Task Decomposition?")#example call


In [ ]:
#Generation part where the llm walks in
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)#so you can use this to give a proper prompt instead of justa  mere string
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

#Now we are gonna create a chain here 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}#the important point to be noted here is that the doccs from the response is given as a parameter internally between the pipes of the chain so it is very simple
    | prompt
    | llm
    | StrOutputParser()#for parsing the string output
)

rag_chain.invoke("What is Task Decomposition?")# when we call this invoke function with a string retreiver("str") will be called and the response is stored in content then question is just assigned to the string
